In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import tensorflow as tf

import sys
sys.path.append('/mnt/c/Users/kheut/code/covid19-forecasting/tf_model_1p5/')

from enum import Enum

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Dense

import tensorflow_probability as tfp
from scipy.stats import beta, truncnorm


# Local imports from model.py, data.py
from model import CovidModel, LogPoissonProb, get_logging_callbacks, Comp, Vax
from model_config import ModelConfig
from data import read_data, create_warmup
#from plots import make_all_plots

import scipy

import matplotlib
import matplotlib.pyplot as plt
plt.rcParams.update({'font.size': 20}) # set plot font sizes

2022-02-28 00:01:59.863291: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-02-28 00:01:59.863325: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
transition_window =10

warmup_start = '20210421'
warmup_end = '20210430'
train_start = '20210501'
train_end = '20210731'
test_start = '20210801'
test_end = '20210831'

state = 'Massachusetts'
state_abbrev = 'MA'

data_dir = '../data'
covid_estim_date = '20210901'
hhs_date = '20210903'
owid_date = '20210903'

# Learning rate
learning_rate = 0.1

In [3]:
df = read_data(data_dir=data_dir,
               covid_estim_date=covid_estim_date,
               hhs_date=hhs_date,
               owid_date=owid_date,
               state=state, state_abbrev=state_abbrev)
vax_statuses = [Vax.yes, Vax.no]
x_train = tf.cast(df.loc[train_start:train_end,'Rt'].values, dtype=tf.float32)
x_test = tf.cast(df.loc[train_start:test_end,'Rt'].values, dtype=tf.float32)


y_train = {}
y_train['G_in'] = tf.cast(df.loc[train_start:train_end,'general_ward_in'], dtype=tf.float32)
y_train['G_count'] = tf.cast(df.loc[train_start:train_end,'general_ward_count'], dtype=tf.float32)
y_train['I_count'] = tf.cast(df.loc[train_start:train_end,'icu_count'], dtype=tf.float32)
y_train['D_in'] = tf.cast(df.loc[train_start:train_end,'deaths_covid'], dtype=tf.float32) + 1

y_test = {}
y_test['G_in'] = tf.cast(df.loc[train_start:test_end,'general_ward_in'], dtype=tf.float32)
y_test['G_count'] = tf.cast(df.loc[train_start:test_end,'general_ward_count'], dtype=tf.float32)
y_test['I_count'] = tf.cast(df.loc[train_start:test_end,'icu_count'], dtype=tf.float32)
y_test['D_in'] = tf.cast(df.loc[train_start:test_end,'deaths_covid'], dtype=tf.float32)

2022-02-28 00:02:01.979156: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:925] could not open file to read NUMA node: /sys/bus/pci/devices/0000:2d:00.0/numa_node
Your kernel may have been built without NUMA support.
2022-02-28 00:02:01.979356: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-02-28 00:02:01.979390: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory
2022-02-28 00:02:01.979417: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot open shared object file: No such file or directory
2022-02-28 00:02:01.979442: W tensorflow/stream_executor/platform/default/dso_loader.cc:6

In [11]:
config = ModelConfig.from_json("/mnt/c/Users/kheut/Downloads/deriv.json")

In [12]:
model = CovidModel([Vax.no, Vax.yes], [Comp.A, Comp.M, Comp.G, Comp.GR, Comp.I, Comp.IR, Comp.D],
                 transition_window,
                config, posterior_samples=1000, debug_disable_theta=False)

pre_training_preds=model.call(x_train)

In [13]:
loss = LogPoissonProb() 
optimizer = tf.keras.optimizers.Adam(
    learning_rate=learning_rate,#beta_1=0.1, beta_2=0.1
)
#loss((y_train['G_count'], y_train['G_in'], y_train['I_count'], y_train['D_in']), pre_training_preds)

In [14]:
logging_callbacks = get_logging_callbacks('/mnt/c/Users/kheut/logs/covid/linear_warm_fixed_loss_2')

In [15]:
model.compile(loss=loss, optimizer=optimizer, run_eagerly=True)
model.fit(x=np.asarray([x_train]), y=np.asarray([(y_train['G_count'], y_train['G_in'], y_train['I_count'], y_train['D_in'])]),
         epochs=2000, batch_size=0,
        callbacks=logging_callbacks)

Epoch 1/2000
G count: -2026.260009765625
G in: -130.9763641357422
I count: -839.614990234375
D in: -9.231890678405762
1/1 [==============================] - 46s 46s/step - loss: 27694.5449
Epoch 2/2000
G count: -1939.3927001953125
G in: -120.39025115966797
I count: -873.853515625
D in: -9.45565414428711
1/1 [==============================] - 45s 45s/step - loss: 25925.5195
Epoch 3/2000
G count: -2041.657958984375
G in: -115.23594665527344
I count: -850.3695068359375
D in: -9.768377304077148
1/1 [==============================] - 44s 44s/step - loss: 24837.7246
Epoch 4/2000
G count: -2013.8651123046875
G in: -118.62358856201172
I count: -867.7510375976562
D in: -10.461901664733887
1/1 [==============================] - 49s 49s/step - loss: 23593.7480
Epoch 5/2000
G count: -1850.7039794921875
G in: -109.7220687866211
I count: -836.6499633789062
D in: -10.274942398071289
1/1 [==============================] - 45s 45s/step - loss: 22753.6074
Epoch 6/2000
G count: -1980.2911376953125
G in: 

KeyboardInterrupt: 

In [ ]:
model.variables

In [ ]:
config.rho_M.value

In [ ]:
isinstance(model.unconstrained_rho_M[0]['loc'], tf.Variable)

In [ ]:
model._constrain_parameters()

In [ ]:
model._sample_and_reparameterize()

In [ ]:
model.rho_M_params

In [ ]:
model.rho_M_samples_constrained

In [ ]:
!pwd

In [9]:
new_config = model.config.update_from_model(model)

In [10]:
new_config.to_json("/mnt/c/Users/kheut/Downloads/deriv.json")

In [16]:
y_train

{'G_in': <tf.Tensor: shape=(92,), dtype=float32, numpy=
 array([70., 71., 65., 50., 57., 70., 74., 64., 71., 48., 44., 58., 49.,
        54., 51., 44., 44., 39., 46., 40., 34., 34., 31., 39., 40., 54.,
        30., 27., 29., 33., 16., 26., 21., 19., 16., 24., 22., 11., 10.,
        26., 18., 13., 19.,  8., 15.,  7., 12., 13., 16., 13., 14., 12.,
        10., 17., 11., 13., 12., 11.,  9., 15., 21.,  9.,  9.,  8.,  8.,
        10.,  4., 15., 14., 13., 23., 17., 18.,  7., 16., 16., 12., 16.,
        15., 12.,  9., 17., 22., 22., 25., 17., 23., 32., 37., 36., 33.,
        46.], dtype=float32)>,
 'G_count': <tf.Tensor: shape=(92,), dtype=float32, numpy=
 array([482., 499., 466., 433., 445., 430., 409., 420., 403., 376., 379.,
        373., 368., 347., 334., 344., 304., 311., 317., 316., 281., 273.,
        296., 240., 258., 257., 242., 223., 225., 225., 212., 201., 193.,
        198., 183., 177., 172., 160., 169., 179., 155., 154., 143., 153.,
        122., 125., 129., 119., 130., 135., 112